# An Overview of Pandas GroupBy

In [1]:
import numpy as np
import pandas as pd

This exercise is inspired by Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653).

In [2]:
df = pd.read_csv("elections.csv")
df.head()

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss


## Grouping a Series by a Series

Let's group the `%` Series by the `Party` Series. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) does that, but what is the object that results?

In [3]:
percent_grouped_by_party = df['%'].groupby(df['Party'])
type(percent_grouped_by_party)

pandas.core.groupby.groupby.SeriesGroupBy

As we see, `percent_grouped_by_party` is **NOT** a DataFrame, Series, or Index. Instead, it's a `SeriesGroupBy` object. A `SeriesGroupBy` consists of `groups`, one for each of the distinct values of the `Party` column. If we ask to see these groups, we'll be able to see which indices in the original DataFrame correspond to each group.

In [4]:
percent_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

The `percent_grouped_by_party` object is capable of making computations across all these groups. For example, if we call the `mean` method of the `SeriesGroupBy` class, we'll get a new `Series` containing the mean of the "Democratic" `Series`, the mean of the "Independent" `Series`, and the mean of the "Republican" `Series`.

In [5]:
percent_grouped_by_party.mean()

Party
Democratic     46.53
Independent    11.30
Republican     47.86
Name: %, dtype: float64

The output of the `mean` methood is a regular ole pandas Series.

In [6]:
type(percent_grouped_by_party.mean())

pandas.core.series.Series

`SeriesGroupBy` objects have many other handy methods, e.g. max and min.

In [7]:
percent_grouped_by_party.max()

Party
Democratic     52.9
Independent    18.9
Republican     58.8
Name: %, dtype: float64

In [8]:
percent_grouped_by_party.min()

Party
Democratic     37.6
Independent     6.6
Republican     37.4
Name: %, dtype: float64

In [9]:
percent_grouped_by_party.size()

Party
Democratic     10
Independent     3
Republican     10
Name: %, dtype: int64

In [10]:
percent_grouped_by_party.first()

Party
Democratic     41.0
Independent     6.6
Republican     50.7
Name: %, dtype: float64

We can iterate over a `SeriesGroupBy` object. Each element is a pair of `(name, group)`, where `name` is a String label for the group, and `group` is a `Series` corresponding to all the values from that group.

In [11]:
from IPython.display import display  # like print, but for complex objects

for name, group in percent_grouped_by_party:
    print('Name:', name)
    print(type(group))
    display(group.head())
    print('\n')

Name: Democratic
<class 'pandas.core.series.Series'>


1     41.0
4     37.6
6     45.6
7     43.0
10    49.2
Name: %, dtype: float64



Name: Independent
<class 'pandas.core.series.Series'>


2      6.6
9     18.9
12     8.4
Name: %, dtype: float64



Name: Republican
<class 'pandas.core.series.Series'>


0     50.7
3     58.8
5     53.4
8     37.4
11    40.7
Name: %, dtype: float64

## Grouping a Series by Multiple Series

We can also group a Series by multiple Series. For example, suppose we want to track all combinations of `{'Democratic', 'Republican', and 'Independent'}` and `{'win', 'loss'}`. 

In [12]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.groups

{('Democratic', 'loss'): Int64Index([1, 4, 6, 13, 15, 21], dtype='int64'),
 ('Democratic', 'win'): Int64Index([7, 10, 17, 19], dtype='int64'),
 ('Independent', 'loss'): Int64Index([2, 9, 12], dtype='int64'),
 ('Republican', 'loss'): Int64Index([8, 11, 18, 20], dtype='int64'),
 ('Republican', 'win'): Int64Index([0, 3, 5, 14, 16, 22], dtype='int64')}

Given this groupby object, we can compute the average percentage earned every time each of the parties won and lost the presidential election. We see that at least between 1980 and 2016, the Republicans have typically lost and won their elections by wider margins.

In [13]:
party_result_mean = percent_grouped_by_party_and_result.mean()
party_result_mean

Party        Result
Democratic   loss      44.850000
             win       49.050000
Independent  loss      11.300000
Republican   loss      42.750000
             win       51.266667
Name: %, dtype: float64

The careful reader will note that the returned object looks a little funny. It seems to have two indexes! If we check the type of this object, we'll see it's just a regular Series.

In [14]:
type(party_result_mean)

pandas.core.series.Series

However if we request to see the index of this `Series`, we see that it is a "MultiIndex", which is a special type of index used for data that is indexed in two or more ways.

In [15]:
party_result_mean.index

MultiIndex(levels=[['Democratic', 'Independent', 'Republican'], ['loss', 'win']],
           labels=[[0, 0, 1, 2, 2], [0, 1, 0, 0, 1]],
           names=['Party', 'Result'])

Selecting an element of a multi-index gives you a series with just an index.

In [16]:
party_result_mean['Republican']

Result
loss    42.750000
win     51.266667
Name: %, dtype: float64

Or you can select both levels at once.

In [17]:
party_result_mean['Democratic', 'win']

49.05

Or you can slice.

In [18]:
party_result_mean[:, "loss"]

Party
Democratic     44.85
Independent    11.30
Republican     42.75
Name: %, dtype: float64

## Grouping a DataFrame by a Series

We can also group an entire dataframe by one or more Series. The result is a `DataFrameGroupBy` object:

In [19]:
everything_grouped_by_party = df.groupby('Party')
everything_grouped_by_party

As in our previous example, this object contains three `group` objects, one for each party label.

In [20]:
everything_grouped_by_party.groups

{'Democratic': Int64Index([1, 4, 6, 7, 10, 13, 15, 17, 19, 21], dtype='int64'),
 'Independent': Int64Index([2, 9, 12], dtype='int64'),
 'Republican': Int64Index([0, 3, 5, 8, 11, 14, 16, 18, 20, 22], dtype='int64')}

Just as with `SeriesGroupBy` objects, we can iterate over a `DataFrameGroupBy` object to understand what is effectively inside.

In [21]:
for n, g in everything_grouped_by_party:
    print('name:', n)
    display(g.head())

name: Democratic


,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win


name: Independent


,Candidate,Party,%,Year,Result
2,Anderson,Independent,6.6,1980,loss
9,Perot,Independent,18.9,1992,loss
12,Perot,Independent,8.4,1996,loss


name: Republican


,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
3,Reagan,Republican,58.8,1984,win
5,Bush,Republican,53.4,1988,win
8,Bush,Republican,37.4,1992,loss
11,Dole,Republican,40.7,1996,loss


And just like `SeriesGroupBy` objects, we can apply methods like `mean` to compute the mean of each group. Since a `DataFrameGroupBy` is linked to the entire original dataframe (instead of to a single column from the dataframe), we calculate a mean for every numerical column. In this example below, we get the mean vote earned (as before), and the mean year (which isn't a useful quantity).

In [22]:
everything_grouped_by_party.mean()

,%,Year
Party,,
Democratic,46.53,1998.000000
Independent,11.30,1989.333333
Republican,47.86,1998.000000


Where did all the other columns go in the mean above? They are *nuisance columns*, which get automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

## Grouping a DataFrame by Multiple Series

DataFrames may also be grouped by multiple series at once. For example, we can repeat what we did with a Series above and group the entire DataFrame by Party and Result. After aggregation, we end up with a DataFrame that has a MultiIndex.

In [23]:
everything_grouped_by_party_and_result=df.groupby([df['Party'], df['Result']])

In [24]:
everything_grouped_by_party_and_result.max()

Candidate     %  Year
Party       Result                      
Democratic  loss     Mondale  48.4  2016
            win        Obama  52.9  2012
Independent loss       Perot  18.9  1996
Republican  loss      Romney  47.2  2012
            win        Trump  58.8  2016

The resulting DataFrame above is pretty strange. We'll observe that Walter Mondale did not run for office in 2016! Make sure you understand why this is happening, as this sort of thing is a common mistake made by people who don't fully understand how pandas works.

**Question**: How do you generate a table like the one above, except that each entry should represent all attributes of the candidate who got the maximum vote in that category? For example, the Republican Win category should feature Reagan in 1984 with 58.8% of the vote. *Hint*: Try sorting.

    .
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    .

In [25]:
(df.sort_values("%", ascending=False).
    groupby([df['Party'], df['Result']]).
    first())

Candidate        Party     %  Year Result
Party       Result                                          
Democratic  loss        Gore   Democratic  48.4  2000   loss
            win        Obama   Democratic  52.9  2008    win
Independent loss       Perot  Independent  18.9  1992   loss
Republican  loss      Romney   Republican  47.2  2012   loss
            win       Reagan   Republican  58.8  1984    win

The result of an aggregation function applied to a DataFrameGroupBy 

## Custom Aggregation Functions

As described above, both `SeriesGroupBy` and `DataFrameGroupBy` objects have lots of handy methods for computing aggregate values for groups, e.g.

In [26]:
percent_grouped_by_party.min()

Party
Democratic     37.6
Independent     6.6
Republican     37.4
Name: %, dtype: float64

In [27]:
everything_grouped_by_party.median()

,%,Year
Party,,
Democratic,48.25,1998
Independent,8.40,1992
Republican,47.55,1998


It turns out that all of these GroupBy methods are just shorthand for a more powerful and universal method of our GroupBy objects called agg. For example, `.min()` is just shorthand for `.agg(min)`, where `min` refers to the function `min`.

In [28]:
percent_grouped_by_party.agg(min)

Party
Democratic     37.6
Independent     6.6
Republican     37.4
Name: %, dtype: float64

Naturally, we can define our own custom aggregation functions. For example, the function below returns the first item in a series.

In [29]:
def average_of_first_and_last(series):
    return (series.iloc[0] + series.iloc[-1])/2

We can supply this function as a custom aggregation function for each series. As you can see, nuisance columns are automatically removed.

In [30]:
percent_grouped_by_party_and_result.agg(average_of_first_and_last)

Party        Result
Democratic   loss      44.60
             win       47.05
Independent  loss       7.50
Republican   loss      42.30
             win       48.40
Name: %, dtype: float64

`agg` is fundamental to our use of GroupBy objects. Indeed, you will rarely call `groupby` without also calling `agg`, at least implicitly. We can summarize the grouping process with the following diagram, inspired by a similar diagram created by DS100 instructor Joey Gonzales. Diagram source at [this link](https://docs.google.com/presentation/d/1FrYg6yd6B-CIgfWLWm4W8vBhfmJ6Qt9dKkN-mlN5AKU/edit#slide=id.g4131093782_0_40).

![groupby](groupby_overview.png) 

The result of calling `groupby` then `agg` on a `Series` is also a `Series`, and the result of calling `groupby` then `agg` on a `DataFrame` is also typically a `DataFrame`, though there are exceptions, e.g. if you use the aggregation function `size`, you will get back a `Series`.

In [31]:
def list_of_first_and_last(series):
    return [series.iloc[0], series.iloc[-1]]

In [32]:
everything_grouped_by_party.agg(list_of_first_and_last)

,Candidate,%,Year,Result
Party,,,,
Democratic,"[Carter, Clinton]","[41.0, 48.2]","[1980, 2016]","[loss, loss]"
Independent,"[Anderson, Perot]","[6.6, 8.4]","[1980, 1996]","[loss, loss]"
Republican,"[Reagan, Trump]","[50.7, 46.1]","[1980, 2016]","[win, win]"


## Using Groups to Filter Datasets

In [33]:
everything_grouped_by_candidate = df.groupby('Candidate')

In [34]:
everything_grouped_by_candidate.filter(lambda subframe: subframe["Year"].max() > 2000)

,Candidate,Party,%,Year,Result
5,Bush,Republican,53.4,1988,win
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
10,Clinton,Democratic,49.2,1996,win
14,Bush,Republican,47.9,2000,win
15,Kerry,Democratic,48.3,2004,loss
16,Bush,Republican,50.7,2004,win
17,Obama,Democratic,52.9,2008,win
18,McCain,Republican,45.7,2008,loss
19,Obama,Democratic,51.1,2012,win


In [35]:
everything_grouped_by_year = df.groupby('Year')

In [36]:
everything_grouped_by_year.filter(lambda subframe: subframe["%"].sum() < 97)

,Candidate,Party,%,Year,Result
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
13,Gore,Democratic,48.4,2000,loss
14,Bush,Republican,47.9,2000,win
21,Clinton,Democratic,48.2,2016,loss
22,Trump,Republican,46.1,2016,win


A visual picture of how filtering works is shown below.

![filter](filter_overview.png)

## Using isin for filtering

This isn't actually related to `groupby` at all, but it seemed natural to bring it up here. 

Sometimes we want to restrict our attention to only rows where certain values appear. For example, we saw last time how we'd look at only rows that contain "Democratic" candidates.

In [37]:
df[df["Party"] == "Democratic"]

,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
13,Gore,Democratic,48.4,2000,loss
15,Kerry,Democratic,48.3,2004,loss
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
21,Clinton,Democratic,48.2,2016,loss


Suppose we wanted to filter such that all Republicans and Democrats appeared. One ugly way to do this would be `df[(df["Party"] == "Democratic") | (df["Party"] == "Republican")]`. However, a better idea is to use the `isin` method.

In [38]:
df[(df["Party"] == "Democratic") | (df["Party"] == "Republican")]

df[df["Party"].isin(["Republican", "Democratic"])]

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
10,Clinton,Democratic,49.2,1996,win
11,Dole,Republican,40.7,1996,loss


## Grouping over a different dimension (bonus topic, less often useful)

Above, we've been grouping data along the rows, using column keys as our selectors.  But we can also group along the columns, for example we can group by how many times the letter a appears in the column name.

In [39]:
grouped = df.groupby(lambda x: x.count('a'), axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group.columns)

0
Index(['%', 'Result'], dtype='object')
1
Index(['Party', 'Year'], dtype='object')
2
Index(['Candidate'], dtype='object')


In [40]:
df.set_index('Candidate').transpose().groupby(len, axis=1).max()

,4,5,6,7,8
Party,Republican,Republican,Republican,Democratic,Independent
%,53.4,52.9,58.8,49.2,6.6
Year,2004,2016,2012,2016,1980
Result,win,win,win,win,loss


## Pivot Tables

Recall from before that we were able to group the % Series by the "Party" and "Result" Series, allowing us to understand the average vote earned by each party under each election result.

In [41]:
percent_grouped_by_party_and_result = df['%'].groupby([df['Party'], df['Result']])
percent_grouped_by_party_and_result.mean()

Party        Result
Democratic   loss      44.850000
             win       49.050000
Independent  loss      11.300000
Republican   loss      42.750000
             win       51.266667
Name: %, dtype: float64

Because we called `groupby` on a Series, the result of our aggregation operation was also a Series. However, I believe this data is more naturally expressed in a tabular format, with Party as the rows, and Result as the columns. The `pivot_table` operation is the natural way to achieve this data format.

In [42]:
df_pivot = df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result', # the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)
df_pivot.head()

Result,loss,win
Party,,
Democratic,44.85,49.050000
Independent,11.30,NaN
Republican,42.75,51.266667


The basic idea is that you specify a Series to be the `index` (i.e. rows) and a Series to be the `columns`. The data in the specified `values` is then grouped by all possible combinations of values that occur in the `index` and `columns` Series. These groups are then aggregated using the `aggfunc`, and arranged into a table that matches the requested `index` and `columns`. The diagram below summarizes how pivot tables are formed. (Diagram inspired by Joey Gonzales). Diagram source at [this link](https://docs.google.com/presentation/d/1FrYg6yd6B-CIgfWLWm4W8vBhfmJ6Qt9dKkN-mlN5AKU/edit#slide=id.g4131093782_0_89).

![groupby](pivot_table_overview.png)

For more on pivot tables, see [this excellent tutorial](http://pbpython.com/pandas-pivot-table-explained.html) by Chris Moffitt.

## List Arguments to pivot_table (Extra)

The arguments to our pivot_table method can also be lists. A few examples are given below.

If we pivot such that only our `columns` argument is a list, we end up with columns that are MultiIndexed.

In [43]:
df.pivot_table(
    index='Result', # the rows (turned into index)
    columns=['Party', 'Candidate'], # the column values
    values='Year', # the field(s) to processed in each group
    aggfunc=list, # group operation
)

Party     Democratic                                                \
Candidate     Carter       Clinton Dukakis    Gore   Kerry Mondale   
Result                                                               
loss          [1980]        [2016]  [1988]  [2000]  [2004]  [1984]   
win              NaN  [1992, 1996]     NaN     NaN     NaN     NaN   

Party                   Independent                        Republican          \
Candidate         Obama    Anderson         Perot                Bush    Dole   
Result                                                                          
loss                NaN      [1980]  [1992, 1996]              [1992]  [1996]   
win        [2008, 2012]         NaN           NaN  [1988, 2000, 2004]     NaN   

Party                                            
Candidate  McCain        Reagan  Romney   Trump  
Result                                           
loss       [2008]           NaN  [2012]     NaN  
win           NaN  [1980, 1984]     NaN  [2016]

If we pivot such that only our `index` argument is a list, we end up with rows that are MultiIndexed.

In [44]:
df.pivot_table(
    index=['Party', 'Candidate'], # the rows (turned into index)
    columns='Result',# the column values
    values='%', # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)

Result                  loss        win
Party       Candidate                  
Democratic  Carter     41.00        NaN
            Clinton    48.20  46.100000
            Dukakis    45.60        NaN
            Gore       48.40        NaN
            Kerry      48.30        NaN
            Mondale    37.60        NaN
            Obama        NaN  52.000000
Independent Anderson    6.60        NaN
            Perot      13.65        NaN
Republican  Bush       37.40  50.666667
            Dole       40.70        NaN
            McCain     45.70        NaN
            Reagan       NaN  54.750000
            Romney     47.20        NaN
            Trump        NaN  46.100000

If we pivot such that only our values argument is a list, then we again get a DataFrame with multi-indexed Columns.

In [45]:
df.pivot_table(
    index='Party', # the rows (turned into index)
    columns='Result',# the column values
    values=['%', 'Year'], # the field(s) to processed in each group
    aggfunc=np.mean, # group operation
)

%                    Year             
Result        loss        win         loss          win
Party                                                  
Democratic   44.85  49.050000  1995.333333  2002.000000
Independent  11.30        NaN  1989.333333          NaN
Republican   42.75  51.266667  2002.000000  1995.333333